In [1]:
import h2o
h2o.init()
from h2o.estimators.word2vec import H2OWord2vecEstimator
import nltk
# nltk.download('stopwords') -- might need if running nltk + stopwords for the first time
from nltk.corpus import stopwords
from h2o.automl import H2OAutoML
import pandas as pd
from sklearn import metrics

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,6 days 22 hours 38 mins
H2O cluster timezone:,Europe/Vienna
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.5
H2O cluster version age:,"14 days, 13 hours and 58 minutes"
H2O cluster name:,H2O_from_python_mackenzie_dm3nzl
H2O cluster total nodes:,1
H2O cluster free memory:,1.289 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [28]:
train_data = h2o.upload_file("/home/mackenzie/Downloads/GermanCleanedTrainingData.csv") # check if path correct 
test_data = h2o.upload_file("/home/mackenzie/Downloads/GermanCleanedTestingData.csv") # check if path correct
STOP_WORDS = set(stopwords.words('german'))
new_stopwords = ['|lbr|', 'ja']
STOP_WORDS = STOP_WORDS.union(new_stopwords)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

In [5]:
print("Break train tweets into sequence of words")
train_words = tokenize(train_data["tweet"])

Break train tweets into sequence of words


In [6]:
print("Break test tweets into sequence of words")
test_words = tokenize(test_data["tweet"])

Break test tweets into sequence of words


In [7]:
print("Build word2vec model for train")
w2v_model_one = H2OWord2vecEstimator(sent_sample_rate = 0.0, epochs = 10)
w2v_model_one.train(training_frame=train_words)

Build word2vec model for train
word2vec Model Build progress: |██████████████████████████████████████████| 100%


In [8]:
print("Build word2vec model for test")
w2v_model_two = H2OWord2vecEstimator(sent_sample_rate = 0.0, epochs = 10)
w2v_model_two.train(training_frame=test_words)

Build word2vec model for test
word2vec Model Build progress: |██████████████████████████████████████████| 100%


In [9]:
# could do synonym check here
#print("Sanity check - find synonyms for the word 'teacher'")
#w2v_model.find_synonyms("teacher", count = 5)

In [15]:
print("Calculate a vector for each train tweet")
tweet_vecs_train = w2v_model_one.transform(train_words, aggregate_method = "AVERAGE")
print(tweet_vecs_train)

Calculate a vector for each train tweet


C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
0.0291293,-0.0563549,-0.00309205,-0.00874928,-0.152444,0.076329,0.0197836,-0.156682,-0.0311497,0.0331275,-0.0514102,-0.00759734,-0.0921724,0.23417,0.0751271,-0.0647309,0.0425146,-0.0339248,-0.109768,0.00189091,-0.163299,0.00792546,-0.133227,-0.0124696,-0.04783,-0.0585472,-0.112219,0.0745889,0.137946,0.0150429,-0.0224849,-0.0227682,0.0939694,-0.029053,-0.0433851,0.132503,0.0283596,-0.0460802,-0.107062,0.0312514,-0.0225912,-0.0677194,0.131732,-0.095006,0.0392678,0.0414449,-0.0547527,0.181339,-0.0807071,0.0836181,0.0363452,-0.131691,-0.143266,-0.0952793,0.110587,0.00497621,0.12193,-0.0160294,-0.108785,-0.0827263,-0.257084,0.0225244,0.0258357,-0.0744465,-0.00954591,-0.0961743,0.0617737,0.0528972,-0.0484603,-0.147192,0.0716757,-0.0445438,-0.0614128,-0.00830384,0.0334794,0.0170624,0.0135088,-0.0303346,-0.0133053,-0.0388469,-0.0988347,0.0793318,-0.0128315,0.0770838,0.101013,0.0268078,0.00296872,-0.0679003,0.000586955,0.118864,-0.0693687,0.134611,-0.044851,0.0467843,-0.0702371,-0.0054008,0.00354876,0.069282,-0.122692,-0.0854221
-0.020289,0.115269,-0.0016796,-0.0155942,0.0243202,-0.0660598,0.0138952,-0.04091,-0.0727459,-0.0206298,0.00772251,-0.0484347,-0.00341719,0.193299,0.246281,0.0759564,0.0430066,-0.105908,-0.160268,-0.038794,-0.229615,-0.0364317,-0.0827229,0.112615,0.0202405,-0.134642,-0.127648,0.0513902,0.0248847,-0.0323642,0.0434213,-0.122694,0.0503713,0.110142,0.0843856,-0.14796,0.144207,0.160098,-0.195316,-0.0866595,0.074925,-0.00350691,0.260468,-0.112127,0.118457,-0.0274727,0.0241238,0.074524,-0.0520599,0.0683611,-0.160389,-0.0189103,-0.121003,0.165582,0.253134,0.0796824,0.186832,-0.0324347,-0.0109388,-0.0373621,-0.249126,-0.031786,0.0882836,0.114533,0.0915308,-0.183541,0.171694,0.0722871,-0.115099,-0.193177,0.107867,0.114245,-0.197168,-0.121551,-0.0254883,0.0653016,-0.183877,-0.0114236,-0.149496,-0.095159,-0.179325,0.0164512,-0.117974,-0.11533,0.22554,-0.177961,0.00298017,-0.179795,0.0260792,0.0559971,-0.233959,0.174775,-0.223886,0.0194388,-0.0910845,0.151882,0.022251,-0.0155746,0.0478414,0.0880277
-0.0253428,0.056287,0.0363188,0.0172438,-0.164082,0.166106,0.0835552,-0.0867098,0.0391808,0.0792269,-0.0338235,0.073912,-0.0455957,0.172466,0.198691,0.101024,0.0017131,0.0213559,-0.213391,-0.0625758,-0.160427,-0.0760626,-0.12163,-0.0435556,-0.0440114,-0.0471189,-0.118607,0.0920026,0.0898629,0.0988965,-0.0275485,-0.0644385,0.0520332,0.0581712,0.0420209,0.0264621,0.0388987,0.0670861,-0.0751736,0.0665786,-0.07797,-0.0263054,0.210391,-0.0714467,0.00182106,0.114163,0.0322469,0.1208,-0.0654437,0.109724,0.0599284,-0.124671,-0.120932,-0.00315898,0.0103579,-0.12295,0.0913121,-0.0261126,-0.0824543,-0.0242883,-0.145794,0.0611703,0.00813424,-0.0688637,0.1508,-0.00360739,0.0921811,0.138241,-0.117111,-0.235415,0.0841958,0.0880682,-0.133225,-0.0321164,0.0641001,-0.053368,-0.0726977,-0.145889,0.0520019,-0.0718572,-0.176017,0.0371041,0.158674,0.0554042,0.00848453,-0.041267,0.0150014,-0.0163349,-0.00370767,0.175144,0.0130477,0.112052,-0.0536757,-0.098824,-0.0131357,0.0453706,-0.0152432,-0.0348165,-0.133764,0.0362345
0.0560245,-0.058401,0.164942,0.0703506,-0.0476878,0.164793,-0.0499999,-0.123435,0.0924741,0.0141972,-0.130689,0.130842,-0.0498206,0.201381,0.120125,-0.0492466,0.0687634,-0.107686,0.0187308,0.0305042,-0.0427067,0.104504,-0.235232,-0.0924881,0.044094,-0.0854511,-0.0261955,0.0378974,0.0188159,0.0677833,0.00812819,0.0502066,0.130502,0.0790179,0.017534,0.0480997,0.0252324,0.0664753,-0.167453,0.0103843,-0.0194064,0.0506962,0.0842848,-0.103715,-0.0548812,0.100362,-0.0418,0.168366,-0.0749091,0.0592347,0.0521234,-0.0215844,-0.239579,-0.0246836,0.0610119,0.08424

In [11]:
print("Calculate a vector for each test tweet")
tweet_vecs_test = w2v_model_two.transform(test_words, aggregate_method = "AVERAGE")

Calculate a vector for each test tweet


In [12]:
# see lines 10 to 12 in the demo for word2Vec for using ml on the vecs and such
#print(tweet_vecs)

In [16]:
# preparing the train and test data sets
# now convert tweet vecs and labels to a pandas dataframe and back to h2o dataframe
# data = job_titles[valid_job_titles,:].cbind(job_title_vecs[valid_job_titles,:])
train = tweet_vecs_train.cbind(train_data["labels"])
test = tweet_vecs_test.cbind(test_data["labels"])


In [17]:
# more on data prep
x = train.columns         # x: A list/vector of predictor column names or indexes. 
                          # This argument only needs to be specified if the user wants to exclude columns from the 
                          # set of predictors. If all columns (other than the response) should be used in prediction, 
                          # then this does not need to be set.

y = "labels"              # This argument is the name (or index) of the response column
x.remove(y)

# need to set train and test
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [18]:
# now the AUTO-ML piece comes in
aml = H2OAutoML(max_models=10) #max_models=10 or 20?, seed?
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [19]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,mean_per_class_error,logloss,rmse,mse
StackedEnsemble_AllModels_AutoML_20190703_140214,0.576218,0.741973,0.501935,0.251938
StackedEnsemble_BestOfFamily_AutoML_20190703_140214,0.577379,0.742217,0.502258,0.252264
GBM_2_AutoML_20190703_140214,0.593774,0.750322,0.501039,0.25104
GBM_1_AutoML_20190703_140214,0.593868,0.752957,0.503496,0.253508
GBM_4_AutoML_20190703_140214,0.598151,0.752089,0.502336,0.252342
GLM_grid_1_AutoML_20190703_140214_model_1,0.598855,0.75415,0.506817,0.256864
GBM_3_AutoML_20190703_140214,0.59952,0.749502,0.500959,0.25096
XGBoost_1_AutoML_20190703_140214,0.600901,0.737306,0.499072,0.249072
DRF_1_AutoML_20190703_140214,0.610473,2.11826,0.528776,0.279604
XGBoost_2_AutoML_20190703_140214,0.614561,0.78781,0.533743,0.284882


In [20]:
# The leader model is stored here
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20190703_140214
No model summary for this model


ModelMetricsMultinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.10087469097584982
RMSE: 0.3176077627764312

ModelMetricsMultinomialGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.25193835746642307
RMSE: 0.5019346147322608


In [21]:
preds = aml.predict(test)
print(preds)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [24]:
var = preds["predict"].cbind(test[y])
print(var)

predict,labels
0,1
0,1
0,0
0,0
0,2
0,2
0,2
0,0
0,2
0,1


In [27]:
# convert to pandas dataframe
y_test = h2o.as_list(test[y], use_pandas=True)
y_pred = h2o.as_list(preds["predict"])
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.accuracy_score(y_test, y_pred))

[[1108    0    0]
 [ 187    0    0]
 [ 349    0    0]]
